In [1]:
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

A=torch.arange(10,dtype=torch.float32).reshape(2,5)
B=A.clone()#类似于numpy的copy
B[0][0]=10
A
import numpy as np
A=np.arange(10,dtype=np.float32).reshape(2,5)
B=A.copy()
B[0][0]=10
A

array([[0., 1., 2., 3., 4.],
       [5., 6., 7., 8., 9.]], dtype=float32)

In [2]:
A=torch.arange(12).reshape(3,4)
A
sum_A=A.sum(axis=1,keepdims=True)#保持维度，便于广播
A/sum_A

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

tensor([[0.0000, 0.1667, 0.3333, 0.5000],
        [0.1818, 0.2273, 0.2727, 0.3182],
        [0.2105, 0.2368, 0.2632, 0.2895]])

In [3]:
A.cumsum(axis=0)#沿x轴累加

tensor([[ 0,  1,  2,  3],
        [ 4,  6,  8, 10],
        [12, 15, 18, 21]])

In [4]:
x=torch.ones(4,dtype=torch.float32)
torch.dot(x,x+1)
torch.mv(A.float(),x)#矩阵向量乘法，数据类型要一致

tensor(8.)

tensor([ 6., 22., 38.])

In [5]:
A = torch.randint(10,(5, 4))#10是上限
B = torch.randint(10,(4, 3))
torch.mm(A, B)#向量乘法


tensor([[ 85,  80,  71],
        [ 69,  87,  71],
        [ 65,  46,  79],
        [ 91,  57,  83],
        [ 87, 106, 112]])

In [6]:
torch.norm(torch.randn(5))#范数

tensor(2.0221)

矩阵求导

标量对向量(列)求导后得到的是行向量(分子布局，每一列的分子不变)

列向量对标量求导的结果还是列向量

列向量对列向量求导的结果是分子布局的矩阵（列数等于X的维数）

pytorch中的一维数组是以列向量为数学计算约定，而已行向量为表示形式的向量。

自动求导

神经网络的训练过程中，前向传播和反向传播交替进行，前向传播通过训练数据和权重参数计算输出结果；反向传播通过导数链式法则计算损失函数对各参数的梯度，并根据梯度进行参数的更新

求导是从外层向内层求的，需要存储之前计算的值

In [7]:
import torch
x=torch.arange(4.)#必须是浮点型才能求导
x.requires_grad_(True)#注意最后的下划线
y=2*torch.dot(x,x)
y
y.backward()#用反向传播函数计算y关于x的梯度
x.grad


tensor([0., 1., 2., 3.], requires_grad=True)

tensor(28., grad_fn=<MulBackward0>)

tensor([ 0.,  4.,  8., 12.])

In [8]:
x.grad.zero_()#默认梯度会累积，重新计算需要清零
y=x.sum()
y
y.backward()
x.grad
#因为y = x.sum() = (x[0][0] + x[0][1] + x[1][0] + x[1][1])
#每个值的梯度都为1

tensor([0., 0., 0., 0.])

tensor(6., grad_fn=<SumBackward0>)

tensor([1., 1., 1., 1.])

In [9]:
x.grad.zero_()
y=x*x#结果是向量，不能直接backward
y.sum().backward()#求和转化为标量再求导([x0*x0,...]->x0*x0+...)
x.grad

tensor([0., 0., 0., 0.])

tensor([0., 2., 4., 6.])

In [10]:
x.grad.zero_()
y=x*x
u=y.detach()#设置u为固定常量
z=u*x
z.sum().backward()
x.grad==u#导数为u

tensor([0., 0., 0., 0.])

tensor([True, True, True, True])

控制流构建函数图

In [11]:
def f(a):
    b=a*2
    while b.norm()<1000:
        b=b*2
    if b.sum()>0:
        c=b
    else:
        c=100*b
    return c
a=torch.randn(size=(),requires_grad=True)#size为空产生标量，标准正态分布
d=f(a)
d.backward()
a.grad==d/a


tensor(True)